In [0]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import VGG16
from keras.optimizers import Adam
import keras.backend as K


#Generating training and testing datasets
train_datagen= ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip= True)
valid_datagen= ImageDataGenerator(rescale=1./255)
size=(128,128)
in_shape=(128,128,3)
train_set= train_datagen.flow_from_directory("train/", 
                                             target_size=size, batch_size=100, class_mode="categorical", 
                                             shuffle=True, seed=20)
valid_set= valid_datagen.flow_from_directory("test/", 
                                             target_size=size, batch_size=100, class_mode="categorical", 
                                             shuffle=False)

#using pre-trained model
base_model=VGG16(input_shape=in_shape, weights="imagenet", include_top=False)

x=base_model.output
x=Conv2D(32, (3,3), activation="relu")(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Flatten()(x)
x=Dense(units=128, activation="relu")(x)
x=Dense(units=64, activation="relu")(x)
x=Dense(units=32, activation="relu")(x)
x=Dense(units=25, activation="softmax")(x)

model=Model(inputs=base_model.inputs, outputs=x)

for layer in model.layers[:16]:
  layer.trainable=False

for layer in model.layers[16:]:
    layer.trainable=True

#Compile and fit the datasets
model.compile(optimizer=Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
step_size_train=train_set.n//train_set.batch_size
step_size_valid=valid_set.n//valid_set.batch_size
model.fit_generator(train_set, steps_per_epoch=step_size_train, epochs=40, 
                    validation_data= valid_set, validation_steps=step_size_valid)

model.save("test3.h5")


Found 44297 images belonging to 25 classes.
Found 12642 images belonging to 25 classes.
Epoch 1/40
442/442 [==============================] - 293s 663ms/step - loss: 1.1606 - accuracy: 0.6600 - val_loss: 0.4623 - val_accuracy: 0.8252
Epoch 2/40
442/442 [==============================] - 290s 656ms/step - loss: 0.4819 - accuracy: 0.8640 - val_loss: 0.2235 - val_accuracy: 0.8786
Epoch 3/40
442/442 [==============================] - 288s 651ms/step - loss: 0.3615 - accuracy: 0.8958 - val_loss: 0.1874 - val_accuracy: 0.8926
Epoch 4/40
442/442 [==============================] - 288s 651ms/step - loss: 0.2800 - accuracy: 0.9192 - val_loss: 0.3363 - val_accuracy: 0.9080
Epoch 5/40
442/442 [==============================] - 288s 652ms/step - loss: 0.2346 - accuracy: 0.9324 - val_loss: 0.2643 - val_accuracy: 0.9144
Epoch 6/40
442/442 [==============================] - 288s 651ms/step - loss: 0.2120 - accuracy: 0.9385 - val_loss: 0.2829 - val_accuracy: 0.9193
Epoch 7/40
442/442 [================

In [0]:
import numpy
import sklearn.metrics as metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from google.colab import files
files.download("test3.h5")

model = load_model('test3.h5')

valid_set= valid_datagen.flow_from_directory("test/", 
                                             target_size=(128,128), batch_size=100, class_mode="categorical", 
                                             shuffle=False)

test_steps_per_epoch = numpy.math.ceil(valid_set.samples / valid_set.batch_size)
valid_set.reset()
predictions = model.predict_generator(valid_set, steps=test_steps_per_epoch)

classes = valid_set.classes[valid_set.index_array]
y_pred=np.argmax(predictions, axis=-1)

test=np.argmax(predictions[0:500], axis=-1)
print(test)

print(confusion_matrix(valid_set.classes[valid_set.index_array],y_pred))

#print(y_pred)

true_classes = valid_set.classes
class_labels = list(valid_set.class_indices.keys())

report = metrics.classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

Found 12642 images belonging to 25 classes.
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 22  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0  0  0  4  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0
  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  6  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6
  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  6  0  4  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0 12  0
  0  0 

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from PIL import Image


model = load_model('/content/25class/test3.h5')
img = Image.open('/content/55f050d5462489a0.jpg')
img = img.resize((128,128))
img = np.array(img)
img = img / 255.0
img = img.reshape(1,128,128,3)
pred_probab=model.predict(img)
result = np.argmax(pred_probab, axis=1)
print(pred_probab)




Using TensorFlow backend.


OSError: ignored

In [0]:
from zipfile import ZipFile
file_name="25class.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')


done


In [0]:
%ls

test/  test3.h5  train/
